In [1]:
import os
import time
import math
import json
import redis
import asyncio
import traceback
from aredis import StrictRedis
from multiprocessing import Process

In [2]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [3]:
from libs.quotation import Quotation
from libs.dailydata import DailyData
from libs.utils import Utils
from libs.cython.compute import compute_stats

In [4]:
policy = asyncio.WindowsSelectorEventLoopPolicy()
asyncio.set_event_loop_policy(policy)

In [5]:
def assist(assist_idx, assist_count):
    
    ar = StrictRedis(host='127.0.0.1', port=6379, db=8)
    data = {}
    task_snapshotting = None
    
    def get_daily_data(date):
        if date not in data:
            dd = DailyData(date, create=False)
            group_size = math.ceil(len(dd.symbols)/assist_count)
            scope = (
                assist_idx*group_size, 
                min((assist_idx+1)*group_size, len(dd.symbols))
            )
            data[date] = (dd, scope)
            
        return data[date]
    
        
    def compute_statistics(date):
#         dd = get_daily_data(date)
#         group_size = math.ceil(len(dd.symbols)/assist_count)
#         scope = (
#             assist_idx*group_size, 
#             min((assist_idx+1)*group_size, len(dd.symbols))
#         )
        dd, scope = get_daily_data(date)
        basics = dd.basics[scope[0]:scope[1], :]
        for _, check_point in enumerate(daily_data.check_points):
            
            snapshot = dd.snapshots[_,scope[0]:scope[1], :]
            statistic = dd.statistic[_,scope[0]:scope[1], :]
            
            time_lapse = dd.get_time_lapse(_)
            ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
            fs5p = dd.snapshots[ma5pm_anchor_idx,scope[0]:scope[1], :]
            
            compute_stats(snapshot, basics, statistic, fs5p, time_lapse)
            
    
    async def snapshotting(date):
        try:
            dd, scope = get_daily_data(date)
#             dd = get_daily_data(date)
#             group_size = math.ceil(len(dd.symbols)/assist_count)
#             scope = (
#                 assist_idx*group_size, 
#                 min((assist_idx+1)*group_size, len(dd.symbols))
#             )

            q = Quotation(symbols=dd.symbols.tolist()[scope[0]:scope[1]])
            basics = dd.basics[scope[0]:scope[1], :]

            for _, check_point in enumerate(dd.check_points):
                if time.time() > check_point:
                    continue

                delay=(check_point-time.time())
                await asyncio.sleep(max(delay,0))
#                 await asyncio.sleep(5)

                try:
                    await q.snapshot(array=dd.snapshots[_,scope[0]:scope[1],:])

                    snapshot = dd.snapshots[_,scope[0]:scope[1], :]
                    statistic = dd.statistic[_,scope[0]:scope[1], :]

                    time_lapse = dd.get_time_lapse(_)
                    ma5pm_anchor_idx = dd.get_ma5pm_anchor_idx(_)
                    fs5p = dd.snapshots[ma5pm_anchor_idx,scope[0]:scope[1], :]

                    compute_stats(snapshot, basics, statistic, fs5p, time_lapse)

                    await ar.publish(f'hq_assist_{assist_idx}_snapshotting', json.dumps({"status":'successful',"idx":_,'check_point':int(check_point)}))
                        
                except Exception as e:
                    error = {
                        "status": 'failed',
                        "idx": _,
                        'check_point':int(check_point),
                        "exception": str(e),
                        'traceback': traceback.format_exc()
                    }
                    await ar.publish(f'hq_assist_{assist_idx}_snapshotting', json.dumps(error))
                    
#                 finally:
#                     if assist_idx == 0:
#                         dd.incremental_save(_)
                        
        finally:
            await q.exit()
    
    
    async def main():
        
        snapshotting_task = None
        
        while True:
            key, value = await ar.brpop(f'hq_assist_{assist_idx}')
            msg = json.loads(value)
            print(f'Assist[{assist_idx}] {msg}')
            
            if msg['command'] == 'snapshotting':
                snapshotting_task = asyncio.create_task(snapshotting(msg['date']))
                
            elif msg['command'] == 'compute_statistics':
                try:
                    compute_statistics(msg['date'])
                    await ar.lpush(f'hq_assist_{assist_idx}_compute_statistics', json.dumps({"status":'success'}))
                except Exception as e:
                    error = {
                        "status": 'failed',
                        "exception": str(e),
                        'traceback': traceback.format_exc()
                    }
                    await ar.lpush(f'hq_assist_{assist_idx}_compute_statistics', json.dumps(error))
                    
            elif msg['command'] == 'incremental_save':
                dd = get_daily_data(msg['date'])
                dd.incremental_save(msg['idx'])

            elif msg['command'] == 'quit':
                    
                if snapshotting_task and snapshotting_task.done() is False:
                    print(f'Assist[{assist_idx}]: snapshotting_task is going to be canceled')
                    snapshotting_task.cancel()
                    
                print(f'Assist[{assist_idx}]: sharedmemory is going to be closed')
                for date in data:
                    data[date].close_sharedmemory()
                break
                
            else:
                pass
            
#     asyncio.run(main())
    asyncio.create_task(main())
    
    return data
    
# if __name__ == '__main__':
#     rd = redis.Redis(host='127.0.0.1', port=6379, db=8)
#     for key in rd.keys():
#         rd.delete(key)

#     ####
#     Utils.update_symbols()
    
#     symbols = Utils.get_running_symbols()
#     assist_count = math.ceil(len(symbols)/800)+1
    
#     rd.set('hq_assist_count', assist_count)

#     processes = []
#     for _ in range(assist_count):
#         proc = Process(target=assist, args=(_, assist_count))
#         processes.append(proc)
#         proc.start()

#     for proc in processes:
#         proc.join()


In [ ]:
# await asyncio.sleep(int(time.mktime(time.strptime(f'{time.strftime("%Y-%m-%d")} 09:13:00', '%Y-%m-%d %H:%M:%S'))) - time.time()) 

rd = redis.Redis(host='127.0.0.1', port=6379, db=8)

symbol_count = len(Utils.get_symbols()) + 10
assist_count = math.ceil(symbol_count/800)+1

rd.set('hq_assist_count', assist_count)

result = []
for _ in range(assist_count):
    data = assist(_, assist_count)
    result.append(data)

In [ ]:
dd.close_sharedmemory()

In [ ]:
rd = redis.Redis(host='127.0.0.1', port=6379, db=8)

In [ ]:

# for key in rd.keys():
#     rd.delete(key)

In [ ]:
Utils.update_symbols()

In [ ]:
symbols = Utils.get_running_symbols()
assist_count = math.ceil(len(symbols)/800)+1
assist_count

In [ ]:
rd.set('hq_assist_count', assist_count)

In [ ]:
result = []
for _ in range(assist_count):
    data = assist(_, assist_count)
    result.append(data)

In [ ]:
date = time.strftime('%Y%m%d')
result[0][date].get_snapshot('13:30:55')

In [8]:
date = time.strftime('%Y%m%d')
date

'20210625'

In [ ]:
dd = DailyData(date)

In [9]:
securities = dd.get_securities()

In [10]:
securities.loc['300057']

name             万顺新材
zt_price         7.14
dt_price         4.76
ma5vpm     121,432.31
mcap            29.70
sum4            23.73
sum9            49.43
sum19           99.32
sum29          146.16
sum59          286.26
Name: 300057, dtype: object

In [12]:
snapshot = dd.get_snapshot('09:45:55')
snapshot

,datetime,timestamp,name,open,close,now,high,low,turnover,volume,bid1,bid1_volume,zhangfu,junjia,liangbi,zhangsu,tingban,ma5
symbol,,,,,,,,,,,,,,,,,,
000001,09:45:55,1624585555,平安银行,23.08,23.08,23.26,23.43,23.06,"7,446,623.00","173,315,578.25",23.25,"4,400.00",0.78,23.27,1.86,0.78,NaN,22.92
000002,09:45:55,1624585555,万 科Ａ,24.40,24.33,24.39,24.49,24.30,"8,493,090.00","207,131,410.03",24.38,300.00,0.25,24.39,1.88,-0.04,NaN,24.33
000004,09:45:55,1624585555,国华网安,18.86,18.86,18.40,19.10,18.36,"1,635,018.00","30,578,430.14",18.39,300.00,-2.44,18.70,2.76,-2.44,NaN,19.60
000005,09:45:55,1624585555,ST星源,1.92,1.92,1.95,1.96,1.91,"3,280,401.00","6,329,516.94",1.94,"111,399.00",1.56,1.93,4.74,1.56,NaN,1.89
000006,09:45:55,1624585555,深振业Ａ,5.04,5.03,5.01,5.04,5.00,"787,000.00","3,947,691.00",5.01,"126,000.00",-0.40,5.02,1.89,-0.60,NaN,5.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688777,09:45:55,1624585555,中控技术,88.60,88.60,88.58,89.85,88.00,"82,432.00","7,296,966.00",88.36,236.00,-0.02,88.52,1.35,-0.02,NaN,90.40
688788,09:45:55,1624585555,科思科技,130.00,130.83,131.00,132.97,130.00,"95,243.00","12,578,420.00",131.01,400.00,0.13,132.07,2.09,0.77,NaN,133.48
688819,09:45:55,1624585555,天能股份,42.05,42.25,41.73,42.30,41.65,"518,955.00","21,741,998.00",41.73,"3,736.00",-1.23,41.90,2.62,-0.76,NaN,41.99


In [ ]:
############################## TEST #####################################

In [ ]:
check_points = Utils.get_check_points()
symbols = Utils.get_running_symbols()

In [ ]:
date = time.strftime('%Y%m%d')

In [ ]:
data = result[0][date]

In [ ]:
q = Quotation(symbols)
snapshot = await q.snapshot()

market_values = await q.get_market_values()

for _, symbol in enumerate(symbols):
    data.basics[_, 0] = market_values[symbol]['zt_price']
    data.basics[_, 1] = market_values[symbol]['dt_price']
    data.basics[_, 3] = market_values[symbol]['mcap']

await q.exit()

In [ ]:
securities = data.get_securities()
securities

In [ ]:
securities.loc['300057']

In [ ]:
ss = data.get_snapshot('13:07:45')
ss

In [ ]:
ss.loc['600354']

In [ ]:
st = time.time()
for _ in range(len(data.check_points)):
    time_lapse = data.get_time_lapse(_)
    ma5pm_anchor_idx = data.get_ma5pm_anchor_idx(_)
    fs5p = data.snapshots[ma5pm_anchor_idx]
    compute_stats(data.snapshots[_], data.basics, data.statistic[_], fs5p, time_lapse)
et = time.time()
et - st

In [ ]:
data.save()